In [8]:
import cv2
import numpy as np
import os
import imutils
from skimage.measure import compare_ssim
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
from collections import OrderedDict
%matplotlib inline

In [9]:
net=cv2.dnn.readNet("../Yolo/yolov3.weights","../Yolo/yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

with open("../Yolo/coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [3]:
ref_dir = '../NMPS-CD/Toys/Ref'
video_dir = '../NMPS-CD/Toys/Track1'
#video_dir = '../NMPS-CD/Road/Track2'
ref_data = []
video_data = []
n_ref_frames=0
n_video_frames=0
i=1
for f1 in os.listdir(ref_dir):
    img = cv2.imread(os.path.join(ref_dir,'Frame'+str(i)+'.jpg'))
    img = cv2.resize(img, (720,1280), interpolation = cv2.INTER_AREA)
    ref_data.append(img)
    n_ref_frames+=1
    i+=1
i=1
for f1 in os.listdir(video_dir):
    img = cv2.imread(os.path.join(video_dir,'Frame'+str(i)+'.jpg'))
    img = cv2.resize(img, (720,1280), interpolation = cv2.INTER_AREA)
    video_data.append(img)
    n_video_frames+=1
    i+=1

In [4]:
CONF_THRESHOLD = 0.7
NMS_THRESHOLD = 0.4
def post_process(frame, outs, conf_threshold, nms_threshold):
    frame_height = frame.shape[0]
    frame_width = frame.shape[1]

    # Scan through all the bounding boxes output from the network and keep only
    # the ones with high confidence scores. Assign the box's class label as the
    # class with the highest score.
    confidences = []
    boxes = []
    class_ids = []
    final_boxes = []
    final_classes=[]
    final_confidences=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > conf_threshold:
                center_x = int(detection[0] * frame_width)
                center_y = int(detection[1] * frame_height)
                width = int(detection[2] * frame_width)
                height = int(detection[3] * frame_height)
                left = int(center_x - width / 2)
                top = int(center_y - height / 2)
                confidences.append(float(confidence))
                boxes.append([left, top, width, height])
                class_ids.append(class_id)

    # Perform non maximum suppression to eliminate redundant
    # overlapping boxes with lower confidences.
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold,
                               nms_threshold)
    for i in indices:
        i = i[0]
        box = boxes[i]
        left = box[0]
        top = box[1]
        width = box[2]
        height = box[3]
        final_boxes.append(box)
        final_classes.append(class_ids[i])
        final_confidences.append(confidences[i])
    return zip(final_boxes, final_confidences, final_classes)

In [5]:
IMG_WIDTH = 416
IMG_HEIGHT = 416
def yolo_get_objects(data):
    objs = []
    for image in data:
        # Create a 4D blob from a frame.
        blob = cv2.dnn.blobFromImage(image, 1 / 255, (IMG_WIDTH, IMG_HEIGHT),[0, 0, 0], 1, crop=False)

        # Sets the input to the network
        net.setInput(blob)

        # Runs the forward pass to get output of the output layers
        outs = net.forward(outputlayers)

        # Remove the bounding boxes with low confidence
        objs.append(list(post_process(image, outs, CONF_THRESHOLD, NMS_THRESHOLD)))
    return objs

In [6]:
%%time
ref_objs = yolo_get_objects(ref_data)
video_objs = yolo_get_objects(video_data)

In [7]:
import copy
import time
output_frames = video_data.copy()
for i in range(0,len(video_objs)):
    output_frames = copy.deepcopy(video_data[i])
    for obj in video_objs[i]:
        left, top, width, height = obj[0]
        cv2.rectangle(output_frames, (left, top), (left + width,top + height),(0, 255, 0),2)

    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.imshow("output", output_frames)
    
    time.sleep(0.1)
    
    keyboard = cv2.waitKey(30) & 0xFF
    if keyboard == 'q' or keyboard == 27:
        break
    
cv2.destroyAllWindows()

In [8]:
video_objs

[[],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [([252, 246, 161, 310], 0.7440266013145447, 2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [([210, 440, 157, 286], 0.7269572615623474, 2)],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [],
 [([94, -11, 220, 364], 0.7860228419303894, 23)],
 [],
 [],
 [],


In [9]:
import copy
import time
output_frames = ref_data.copy()
for i in range(0,len(ref_objs)):
    output_frames = copy.deepcopy(ref_data[i])
    for obj in ref_objs[i]:
        left, top, width, height = obj[0]
        cv2.rectangle(output_frames, (left, top), (left + width,top + height),(0, 255, 0),2)

    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.imshow("output", output_frames)
    
    time.sleep(0.075)
    
    keyboard = cv2.waitKey(30) & 0xFF
    if keyboard == 'q' or keyboard == 27:
        break
    
cv2.destroyAllWindows()

In [9]:
import copy
import time
output_frames = video_data.copy()
for i in range(0,len(video_objs)):
    output_frames = copy.deepcopy(video_data[i])
    for obj in video_objs[i]:
        left, top, width, height = obj[0]
        cv2.rectangle(output_frames, (left, top), (left + width,top + height),(0, 255, 0),2)

    cv2.namedWindow("output", cv2.WINDOW_NORMAL)
    cv2.imshow("output", output_frames)
    
    time.sleep(0.075)
    
    keyboard = cv2.waitKey(30) & 0xFF
    if keyboard == 'q' or keyboard == 27:
        break
    
cv2.destroyAllWindows()

In [23]:
classes

['person',
 'bicycle',
 'car',
 'motorbike',
 'aeroplane',
 'bus',
 'train',
 'truck',
 'boat',
 'traffic light',
 'fire hydrant',
 'stop sign',
 'parking meter',
 'bench',
 'bird',
 'cat',
 'dog',
 'horse',
 'sheep',
 'cow',
 'elephant',
 'bear',
 'zebra',
 'giraffe',
 'backpack',
 'umbrella',
 'handbag',
 'tie',
 'suitcase',
 'frisbee',
 'skis',
 'snowboard',
 'sports ball',
 'kite',
 'baseball bat',
 'baseball glove',
 'skateboard',
 'surfboard',
 'tennis racket',
 'bottle',
 'wine glass',
 'cup',
 'fork',
 'knife',
 'spoon',
 'bowl',
 'banana',
 'apple',
 'sandwich',
 'orange',
 'broccoli',
 'carrot',
 'hot dog',
 'pizza',
 'donut',
 'cake',
 'chair',
 'sofa',
 'pottedplant',
 'bed',
 'diningtable',
 'toilet',
 'tvmonitor',
 'laptop',
 'mouse',
 'remote',
 'keyboard',
 'cell phone',
 'microwave',
 'oven',
 'toaster',
 'sink',
 'refrigerator',
 'book',
 'clock',
 'vase',
 'scissors',
 'teddy bear',
 'hair drier',
 'toothbrush']

In [10]:
import cv2
import numpy as np
import os
import imutils
from skimage.measure import compare_ssim
import matplotlib.pyplot as plt
from scipy.spatial import distance as dist
from collections import OrderedDict
%matplotlib inline

In [11]:
net=cv2.dnn.readNet("../Yolo/yolov3.weights","../Yolo/yolov3.cfg")
net.setPreferableBackend(cv2.dnn.DNN_BACKEND_OPENCV)
net.setPreferableTarget(cv2.dnn.DNN_TARGET_OPENCL)
layer_names = net.getLayerNames()
outputlayers = [layer_names[i[0]-1] for i in net.getUnconnectedOutLayers()]

with open("../Yolo/coco.names","r") as f:
    classes = [line.strip() for line in f.readlines()]

In [12]:
import time
cap=cv2.VideoCapture(0) #0 for 1st webcam
font = cv2.FONT_HERSHEY_PLAIN
starting_time= time.time()
frame_id = 0
colors= np.random.uniform(0,255,size=(len(classes),3))

while True:
    _,frame= cap.read() # 
    frame_id+=1
    
    height,width,channels = frame.shape
    #detecting objects
    blob = cv2.dnn.blobFromImage(frame,0.00392,(320,320),(0,0,0),True,crop=False) #reduce 416 to 320    

        
    net.setInput(blob)
    outs = net.forward(outputlayers)
    #print(outs[1])


    #Showing info on screen/ get confidence score of algorithm in detecting an object in blob
    class_ids=[]
    confidences=[]
    boxes=[]
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            if confidence > 0.3:
                #onject detected
                center_x= int(detection[0]*width)
                center_y= int(detection[1]*height)
                w = int(detection[2]*width)
                h = int(detection[3]*height)

                #cv2.circle(img,(center_x,center_y),10,(0,255,0),2)
                #rectangle co-ordinaters
                x=int(center_x - w/2)
                y=int(center_y - h/2)
                #cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),2)

                boxes.append([x,y,w,h]) #put all rectangle areas
                confidences.append(float(confidence)) #how confidence was that object detected and show that percentage
                class_ids.append(class_id) #name of the object tha was detected

    indexes = cv2.dnn.NMSBoxes(boxes,confidences,0.4,0.6)


    for i in range(len(boxes)):
        if i in indexes:
            x,y,w,h = boxes[i]
            label = str(classes[class_ids[i]])
            confidence= confidences[i]
            color = colors[class_ids[i]]
            cv2.rectangle(frame,(x,y),(x+w,y+h),color,2)
            cv2.putText(frame,label+" "+str(round(confidence,2)),(x,y+30),font,1,(255,255,255),2)
            

    elapsed_time = time.time() - starting_time
    fps=frame_id/elapsed_time
    cv2.putText(frame,"FPS:"+str(round(fps,2)),(10,50),font,2,(0,0,0),1)
    
    cv2.imshow("Image",frame)
    key = cv2.waitKey(1) #wait 1ms the loop will start again and we will process the next frame
    
    if key == 27: #esc key stops the process
        break;
    
cap.release()    
cv2.destroyAllWindows()